## Calculating Credit Valuation Adjustment (CVA) for OTC derivatives portfolio

This notebook demonstrates how to access portolio from MARVAL and use the async-cva end point of Instrument Pricing Analytics (IPA) in the [LSEG Data Platform](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-cva-definition) to get the Counterparty Valuation Adjustment (CVA) report.

Set up new Portfolio in MARVAL before starting this codebook project:
- Go to [MARVAL Trade Import](https://workspace.refinitiv.com/web/Apps/MARVALTradeImport), use the Excel template included in the CVA codeboook project. You need to download it first.
- Create a new portfolio from MARVAL Trade Import.
- Select Swaps as instrument type, browse your downloaded Excel template and upload all instruments in your portfolio.

#### Learn more

Please visit the [LSEG Developer Community Portal](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis/documentation#ipa-financial-contracts-cva-definition) to access the API documentation on CVA and much more.

You may also visit the [API Playground](https://apidocs.refinitiv.com/Apps/ApiDocs) to get more examples, find  end points and download Swagger files

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on the [LSEG Developer Community Q&A Forum](https://community.developers.refinitiv.com/spaces/231/index.html). The LSEG Developer Community will be very pleased to help you. 

## Imports

In [1]:
import pandas as pd
from modules.PortfolioLoader import PortfolioLoader
from modules.EntityManager import CounterpartyManager, CounterpartyParams, SelfEntity
from modules.CVACalculator import CVACalculator, PricingParameters
from modules.CVAVisualizer import CVAVisualizer

## Loading porfolios and instruments

In [2]:
portfolio_loader = PortfolioLoader()
portfolios = portfolio_loader.show_portfolios()
portfolios

,scope,code,baseCurrency
0,GESG1-186010,CVASAMPLE,USD


In [3]:
portfolio = portfolio_loader.load_portfolio('CVASAMPLE')

### View portfolio transactions

In [4]:
transactions = portfolio_loader.load_transactions()
transactions

,Instrument ID,Counterparty
0,LUID_0000AGMY,8589934316
1,LUID_0000AGN1,8589934333
2,LUID_0000AGMZ,4295875726
3,LUID_0000AGN0,8589934316


### Load instruments from MARVAL

In [5]:
# if you want to load the portfolio as of a date pass the date to the load_instruments function. eg. load_instruments('2024-10-11')
instruments = portfolio_loader.load_instruments() 
# instruments

In [6]:
instruments_brief = pd.DataFrame([{
    'instrumentType': instrument['instrumentType'],
    'csaTag': instrument['csaTag'],
    "instrumentTag": instrument['instrumentDefinition']['instrumentTag']
} for instrument in instruments])

instruments_brief

,instrumentType,csaTag,instrumentTag
0,Swap,CitigroupInc,LUID_0000AGN0
1,Swap,UniCreditSpA,LUID_0000AGMZ
2,Swap,CitigroupInc,LUID_0000AGMY
3,Swap,BarclaysPLC,LUID_0000AGN1


## Loading and associating risk profile to each counterparty

Set credit risk profile on all your counterparties. You can use the [MARVAL Counterparty Editor](https://workspace.refinitiv.com/web/rap/marvalcounterpartyeditor) to search for Credit Curve. Use the CounterpartyManager object below to get current counterparties and set credit risk profiles.

In [7]:
counterparty_manager =  CounterpartyManager()

In [8]:
counterparties = counterparty_manager.get_counterparties(portfolio.counterparties)
counterparties.to_df()

,name,assigned_to
8589934316,CitigroupInc,LUID_0000AGMY
8589934333,BarclaysPLC,LUID_0000AGN1
4295875726,UniCreditSpA,LUID_0000AGMZ


In [9]:
counterparty_manager.enhance_counterparty_data(
            reference_entity="8589934333", 
            enhancement_data = CounterpartyParams(
                                    curve_id="0#BCSBEUAMRBMK=", 
                                    collateral_currency="EUR",
                                    recovery_deal=40, 
                                    recovery_instrument=40
                                    ))

counterparty_manager.enhance_counterparty_data(
            reference_entity="8589934316", 
            enhancement_data = CounterpartyParams(
                                    curve_id="0#CUSAXRBMK=", 
                                    collateral_currency="USD",
                                    recovery_deal=40, 
                                    recovery_instrument=40))

counterparty_manager.enhance_counterparty_data(
            reference_entity="4295875726",
            enhancement_data = CounterpartyParams(
                                    curve_id="0#UNICEUAMRBMK=", 
                                    collateral_currency="EUR",
                                    recovery_deal=40, 
                                    recovery_instrument=40,
                                    ))



In [10]:
counterparties.to_df()

,name,assigned_to,CurveID,CollateralCurrency,RecoveryInstrument,RecoveryDeal
8589934316,CitigroupInc,LUID_0000AGMY,0#CUSAXRBMK=,USD,40,40
8589934333,BarclaysPLC,LUID_0000AGN1,0#BCSBEUAMRBMK=,EUR,40,40
4295875726,UniCreditSpA,LUID_0000AGMZ,0#UNICEUAMRBMK=,EUR,40,40


In [11]:
portfolio_loader.set_counterparties(counterparties)

## CVA Calculation


This sample value an existing portfolio of Swaps. The 'fields' requested are:
- CVA Bilateral and Unilateral (BilateralCvaInReportCcy, UnilateralCvaInReportCcy)
- CVA Bilateral and Unilateral (BilateralDvaInReportCcy, UnilateralDvaInReportCcy)
- MarketValueInReportCcy (MarketValueInReportCcyArray)

Use CVACalculator object below to calculate CVA

In [12]:
cva_calculator = CVACalculator()

##### At this point, Self counterparty should set to value the Debt Valuation Adjustment (DVA)

In [13]:
cva_result = cva_calculator.calculate_cva(
        portfolio=portfolio,
        pricing_parameters = PricingParameters(
                    valuation_date="2024-12-13",
                    simulation_count=1000,
                    self_reference_entity="LSEG",
                    self_recovery_rate_percent=40.0,
                    report_ccy="GBP",
                    numeraire_type="Cash"
            ),
        entity = SelfEntity(
                    name="LSEG",
                    collateral_currency="GBP",
                    recovery_instrument=40.0,
                    recovery_deal=40.0,
                    curve_id="0#BBBGBPBMK="
            )
        )

cva_result

,CsaTag,BilateralCvaInReportCcy,BilateralDvaInReportCcy,UnilateralCvaInReportCcy,UnilateralDvaInReportCcy,MarketValueInReportCcyArray,Exposure,PotentialFutureExposure,Allocations,ErrorMessage
0,CitigroupInc,3896.311264,2018.471413,4046.650026,2042.624510,"[16115.805131814253, -47985.22178132968]","[{'exposureDate': '2024-12-27', 'expectedNegat...","[{'exposureDate': '2024-12-27', 'expectedNegat...","[{'instrumentTag': 'LUID_0000AGN0', 'instrumen...",
1,BarclaysPLC,3938.417961,333.449270,4007.938734,335.734216,[71172.06233058788],"[{'exposureDate': '2025-08-29', 'expectedNegat...","[{'exposureDate': '2025-08-29', 'expectedNegat...","[{'instrumentTag': 'LUID_0000AGN1', 'instrumen...",
2,UniCreditSpA,16789.199401,11924.265986,17558.012263,12068.644337,[-225857.39918931472],"[{'exposureDate': '2025-09-08', 'expectedNegat...","[{'exposureDate': '2025-09-08', 'expectedNegat...","[{'instrumentTag': 'LUID_0000AGMZ', 'instrumen...",


In [15]:
# to view the CVA universe, comment out run the function below.
# cva_calculator.cva_universe

In [ ]:
# to view the CVA counterparties, comment out run the function below.
# cva_calculator.cva_counterparties

## CVA Visualisation

Use the CVAVisualizer object to view CVA results.

In [14]:
visualizer = CVAVisualizer(cva_result)

#### View allocations for all counterparties

In [15]:
visualizer.plot_allocations()

#### View allocations per counterparty

In [16]:
visualizer.plot_allocations(csa_tag = "CitigroupInc")

#### View market values for all counterparties

In [17]:
visualizer.plot_market_values()

#### View market values per counterparty

In [18]:
visualizer.plot_market_values(csa_tag="CitigroupInc")

#### View Potential Future Exposure

In [19]:
visualizer.plot_potential_future_exposure(csa_tag='CitigroupInc')

#### View Negative and Positive Exposure

In [20]:
visualizer.plot_exposure(csa_tag='CitigroupInc')